# Segundo Trabalho de Inteligência Artificial e Sistemas Inteligentes

Leandro Furlam Turi

## Load packages

In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import cross_validate, GridSearchCV, RepeatedStratifiedKFold

## Implementation of some methods

### Probabilistic OneR

In [ ]:
'''OneR Algorithm
For each predictor,
     For each value of that predictor, make a rule as follows;
           Count how often each value of target (class) appears
           Find the most frequent class
           Make the rule assign that class to this value of the predictor
     Calculate the total error of the rules of each predictor
Choose the predictor with the smallest total error.
'''

class OneRProbabilistic(BaseEstimator, ClassifierMixin):
    def __init__(self, disc=True):
        self._disc = disc

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)

        self.X_ = X
        self.y_ = y

        # Discretization
        self.__est = KBinsDiscretizer(n_bins=2*self.n_classes_, encode='ordinal', strategy='kmeans') if self._disc else None
        X_disc = self.__est.fit_transform(X) if self._disc else X

        # Contingency tables
        contingency_tb = [pd.crosstab(x, y, rownames=['data'], colnames=['target']) for x in X_disc.T]
        # Feature with greater power differentiation
        self.__great_diff = np.argmax([tb.max(axis=1).sum() for tb in contingency_tb])
        # Class distribution of each characteristic value
        self.__dist_class = contingency_tb[self.__great_diff].div(contingency_tb[self.__great_diff].sum(axis=1), axis=0)

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Discretization
        X_disc = self.__est.transform(X) if self._disc else X

        # Input validation
        X_disc = check_array(X_disc)

        # roulette taking into account the distribution of classes
        predicted = [self.__dist_class.columns[(self.__dist_class.loc[x].cumsum() >= np.random.rand()).idxmax()] for x in X_disc[:,self.__great_diff]]
        return np.array(predicted)
    
    def get_params(self, deep=False):
        return {'disc': self._disc}

### KCentroids

In [ ]:
# KCentroids

'''KCentroids
The KCentroides classifier uses a grouping algorithm to define K groups of examples from each class in the training base.
Assuming that a database has ncl classes, the KCentroides algorithm initially forms K * ncl groups, with K groups in each of the ncl classes.
Then, the centroid of each group is calculated and this centroid is associated with the class of the group from which it was generated.
The hyperparameter method has the value of K.
To perform a classification, KCentroides checks which centroid is closest to the element to be classified and returns to its class.
'''

class KCentroids(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator='kmeans', argv=None, n_clusters=8):
        self._estimator = estimator
        self._n_clusters = n_clusters
        self._argv = argv
        if estimator == 'kmeans':
            self._est = KMeans(n_clusters)
        elif estimator == 'ga':
            self._est = GA(n_clusters, argv)
        else:
            raise Exception('Estimator not definied!')

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = np.unique(y)

        self.X_ = X
        self.y_ = y

        # Geting centroids with estimator
        self.__groups = [self._est.fit(X[y == c]).cluster_centers_ for c in self.classes_]

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        predicted = [self.classes_[np.argmin([min([np.linalg.norm(x - c) for c in g]) for g in self.__groups])] for x in X]
        return np.array(predicted)

    def get_params(self, deep=False):
        return {'estimator': self._estimator,
                'n_clusters': self._n_clusters,
                'argv': self._argv}

#### Genetic Algorithm (from T1)

In [ ]:
class GA():
    '''Genetic Algorithm
    The initial population is constructed by hill climbing algorithm. 
    The selection mechanism is given by tournament, where a number of candidates for parents are put forward and only the best two are selected to generate offspring.
    Since the crossing mechanism selects only two individuals, this step is repeated pop_size/2 times, so that all individuals have an equal chance of performing the recombination.
    The combination mechanism used was also highly inspired by nature, to which the father donates a sperm that will be fertilized in an egg donated by the mother.
    Technically, the father donates a randomly chosen cluster and it is inserted into the mother. 
    Then, the items present in the cluster donated by the father are removed from the mother cluster generating a new individual. 
    If it occurs to result in the child more clusters than necessary, those with the least amount of items are grouped; otherwise, the cluster with the largest amount are separated in half.
    This technique was adopted so that good groupings can be maintained.
    The mutation mechanism applied is strong: if the individual is selected, he is discarded and a new individual is inserted into the population.
    Note, therefore, that the mutation occurs more in the population than in the individual.
    As for the offspring, only the pop_size best individuals are allowed to enter a new generation.
    As a stopping criterion, maximum number of iterations, maximum processing time and whether the population has converged, that is, if all individuals are identical, are used.
    '''
    def __init__(self, n_clusters, argv):
        self.k_ = n_clusters
        self.pop_size_ = argv['pop_size']
        self.iter_max_ = argv['iter_max']
        self.cross_ratio_ = argv['cross_ratio']
        self.mut_ratio_ = argv['mut_ratio']
        self.max_time_ = argv['max_time']

    def fit(self, X):
        # Check that X have correct shape
        X = check_array(X)

        self.X_ = X

        # Geting centroids with estimator
        solution = self.genetic(X, self.k_, self.pop_size_, self.iter_max_, 
                                       self.cross_ratio_, self.mut_ratio_, self.max_time_)
        self.cluster_centers_ = solution['cluster_centers_'],
        self.sse_ = solution['sse_'],
        self.clusters_ = solution['clusters_']

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        predicted = [self.classes_[np.argmin([np.linalg.norm(x - g) for g in self.__solution['cluster_centers_']])] for x in X]
        return np.array(predicted)


    def __available_closest_items(self, cluster, available_items, n):
        # n items more closet to centroid of cluster
        n = min(n, len(available_items))
        evaluations = [self.__evaluate_cluster(cluster + [i]) for i in available_items]
        closest = np.argpartition([e['sum_dist'] for e in evaluations], range(n))[:n]
        return closest

    def hill_climbing(self, items, k):
        num_best = k
        available_items = list(range(len(items)))
        np.random.shuffle(available_items)
        # choose the firsts items of clusters randomly
        clusters = [[items[available_items.pop()]] for _ in range(k)]

        current_cluster = 0
        while available_items:
            closest = self.__available_closest_items(clusters[current_cluster], [items[p] for p in available_items], num_best)
            choiced = available_items[closest[np.random.randint(len(closest))]]
            clusters[current_cluster].append(items[choiced])
            available_items.remove(choiced)
            current_cluster = (current_cluster + 1) % k

        return clusters

    def __evaluate_cluster(self, cluster):
        # euclidean norm
        items = [s['coord'] for s in cluster]
        mu = np.average(items, axis=0) # centroid
        
        norm2 = [np.linalg.norm(i-mu) for i in items]
        return {'sum_dist': sum(norm2), 'dist': norm2, 'mu': mu.tolist(), 'items': cluster}

    def __evaluate_clusters(self, clusters):
        states = [self.__evaluate_cluster(cluster) for cluster in clusters]
        return states

    def __objective_function(self, states):
        # SSE metric
        sse = sum([state['sum_dist'] for state in states])
        return sse

    def __random_state(self, states):
        n = len(states)-1
        if n <= 1:
            return states[0]

        index = np.random.randint(0, n)
        return states[index]

    def __initial_population(self, n, items, k):
        pop = [self.__evaluate_clusters(self.hill_climbing(items, k)) for _ in range(n)]
        return pop

    def __selection(self, population, n, n_tournament):
        # by tournament
        N = len(population)
        if n >= N:
            return population
        if n_tournament > N:
            n_tournament = N

        selecteds = np.random.choice(range(0, len(population)), size=n_tournament, replace=False)
        selecteds.sort()
        values = [self.__objective_function(population[i]) for i in selecteds]
        pos_ordered = np.argpartition(values, range(n))[-n:]
        return [population[p] for p in pos_ordered]

    def __crossover(self, dad, mom):
        # son product of an egg fertilized by a sperm
        sperm = self.__random_state(dad)['items']
        # take all groups that not contain the sperm, and remove it too
        egg = [e for e in [[n for n in m['items'] if n not in sperm] for m in mom] if e]
        son = egg + [sperm]

        k = len(dad)
        while len(son) > k: # concatenate smaller groups
            smallers = np.argpartition([len(s) for s in son], range(2))[:2]
            son[smallers[0]] += son[smallers[1]]
            del son[smallers[1]]

        while len(son) < k: # separate larger groups
            larger = np.argmax([len(s) for s in son])
            half = len(son[larger])//2
            son += [son[larger][:half]] + [son[larger][half:]]
            del son[larger]

        return self.__evaluate_clusters(son)

    def __mutation(self, items, k):
        # new individual
        return self.__evaluate_clusters(self.hill_climbing(items, k))

    def __convergent(self, population):
        clusters_0 = [sorted([i['id'] for i in p['items']]) for p in population[0]]
        clusters = [[sorted([i['id'] for i in clusters['items']]) for clusters in states] for states in population]
        return all(c == clusters_0 for c in clusters)

    def __evaluate_population(self, population):
        return sum([self.__objective_function(p) for p in population], [])

    def __offspring(self, population, n):
        best_index = np.argpartition([sum([q['sum_dist'] for q in p]) for p in population], range(n))[:n]
        return [population[i] for i in best_index]

    def genetic(self, X, k, pop_size, iter_max, cross_ratio, mut_ratio, max_time):
        items = [{'id': x, 'coord': y} for x, y in zip(range(len(X)), X)]
        n_tournament = 3
        half_pop = pop_size//2
        pop = self.__initial_population(pop_size, items, k)
        iter = 0
        end = 0

        start = time.process_time()
        while True:
            new_pop = pop.copy()
            for _ in range(half_pop): # everyone can cross
                if np.random.uniform(0, 1, 1) <= cross_ratio:
                    parents = self.__selection(pop, 2, n_tournament)
                    new_pop.append(self.__crossover(parents[0], parents[1]))
                if np.random.uniform(0, 1, 1) <= mut_ratio:
                    new_pop.append(self.__mutation(items, k))
            pop = self.__offspring(new_pop, pop_size)
            val_pop = self.__evaluate_population(pop)

            iter += 1
            end = time.process_time()
            if iter >= iter_max:
                br = 'iteration'
                break
            if end-start > max_time:
                br = 'time'
                break
            if self.__convergent(pop):
                br = 'convergence'
                break

        best_individual = self.__offspring(pop, 1)[0]
        return {'cluster_centers_': [b['mu'] for b in best_individual],
                'sse_': [b['sum_dist'] for b in best_individual],
                'clusters_': [[c['coord'] for c in b['items']] for b in best_individual]}


## Search parameters

In [ ]:
models = {
    'ZeroR': DummyClassifier(strategy='most_frequent'),
    'Random': DummyClassifier(strategy='uniform'),
    'Stratified Random': DummyClassifier(strategy='stratified'),
    'OneR': OneRProbabilistic(),
    'Gaussian Naive Bayes': GaussianNB(),
    'KmeansCentroids': KCentroids('kmeans'),
    'KGACentroids': KCentroids('ga', {'pop_size':50, 'iter_max':100,'cross_ratio':0.75, 'mut_ratio':0.2, 'max_time':1}),
    'Knn': KNeighborsClassifier(weights='uniform'),
    'DistKnn': KNeighborsClassifier(weights='distance'),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
}

parameters = {
    'KmeansCentroids': {'n_clusters': [1, 3, 5, 7]},
    'KGACentroids': {'n_clusters': [1, 3, 5, 7]},
    'Knn': {'n_neighbors': [1, 3, 5, 7]},
    'DistKnn': {'n_neighbors': [1, 3, 5, 7]},
    'Decision Tree': {'max_depth': [None, 3, 5, 10]},
    'Random Forest': {'n_estimators': [10, 20, 50, 100]},
}

## Experimental

In [ ]:
def experimental(base, filename):
    X, y = iris.data, iris.target
    print()
    print(filename)
    ans = {}
    for c in ['ZeroR', 'Random', 'Stratified Random', 'OneR', 'Gaussian Naive Bayes']:
        start = time.process_time()
        pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', models[c])])
        ans[c] = cross_validate(pipeline, X, y, 
                                cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234), scoring='accuracy')
        print('Elapsed time of {} is {:.6f} seconds.'.format(c, time.process_time() - start))

    for c in ['KmeansCentroids', 'KGACentroids', 'Knn', 'DistKnn', 'Decision Tree', 'Random Forest']:
        start = time.process_time()
        clf = Pipeline([('transformer', StandardScaler()), 
                        ('estimator', GridSearchCV(models[c], param_grid=parameters[c], scoring='accuracy', 
                                                   cv=RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=36851234)))]).fit(X, y) # Grid search
        pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', clf)])
        ans[c] = cross_validate(pipeline, X, y, 
                                cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234), scoring='accuracy')
        print('Elapsed time of {} is {:.6f} seconds.'.format(c, time.process_time() - start))

    pickle.dump(ans, open(filename, "wb" ))

### Execution

In [ ]:
iris = datasets.load_iris()
digits = datasets.load_digits()
wine = datasets.load_wine()
cancer = datasets.load_breast_cancer()

for k, base in {'iris':iris, 'digits':digits, 'wine':wine, 'cancer':cancer}.items():
    experimental(base, k)

### Results

In [ ]:
import pickle

filename = "iris"
ans = pickle.load(open(filename, "rb" ))

In [ ]:
DF = pd.DataFrame()
for a in ans.keys():
    DF[a] = ans[a]['test_score']
DF.head()

In [ ]:
pd.DataFrame({'mean': DF.mean(), 'std': DF.std(), 
              'confit_inf': DF.apply(lambda x: st.t.interval(0.95, len(x)-1, loc=np.mean(x), scale=st.sem(x))[0]),
              'confit_sup': DF.apply(lambda x: st.t.interval(0.95, len(x)-1, loc=np.mean(x), scale=st.sem(x))[0])})

In [ ]:
plt.figure()
sns.boxplot(data=DF)
plt.xticks(rotation=90)
plt.ylabel('Value')
plt.tight_layout()
plt.show()

In [ ]:
import scipy.stats as stats
M = [[-1]*len(ans) for _ in range(len(ans))]

for i in range(len(ans)):
    for j in range(i+1, len(ans)):
        # Calculate the t-test on TWO RELATED samples of scores, a and b.
        M[i][j] = stats.ttest_rel(DF.iloc[:,i], DF.iloc[:,j]).pvalue
        if any((DF.iloc[:,i] - DF.iloc[:,j]) != 0):
            # Calculate the Wilcoxon signed-rank test.
            M[j][i] = stats.wilcoxon(DF.iloc[:,i], DF.iloc[:,j]).pvalue
pd.DataFrame(M, index=DF.columns, columns=DF.columns)
# Reject when p-value is greater then 0.05